## Singular Value Decomposition

Singular Value Decomposition (SVD) is a crucial technique for matrix factorization. It's used to simplify large datasets into their low-rank approximations, implying that a few key patterns can explain the complex, high-dimensional data.

SVD is unique in that it is guaranteed to exist for every matrix. It's also numerically stable and provides a hierarchical data representation.

$ X = \begin{bmatrix} 
        |   & |   & \vdots & |  \\ 
        x_1 & x_2 & \dots & x_m \\
        |   & |   & \vdots & | 
       \end{bmatrix}^{m \times n} $
       
The matrix $X$ may be represented by the product of matrices $ U\Sigma V^T$ 
       
$ U\Sigma V^T = \begin{bmatrix}
                |   &  |   &  \vdots & |  \\ 
                u_1 &  u_2 &  \dots  & u_n \\
                |   &  |   &  \vdots & |                 
                \end{bmatrix}
               \begin{bmatrix}
                  \sigma_1 &   0        &    0      \\ 
                  0        &   \ddots   &    0      \\
                  0        &   0        & \sigma_m \\
                  0        &   0        &    0     \\
                  \vdots   &   \vdots   &  \vdots  
                \end{bmatrix} 
                \begin{bmatrix}
                |   &  |   &  \vdots & |  \\ 
                v_1 &  v_2 &  \dots  & v_n \\
                |   &  |   &  \vdots & |                 
                \end{bmatrix}^{T}$               

The columns of  $U$ ($u_i$) are similar to the columns of $X$ ($x_i$) and can be considered as the eigenvectors of $X$. Eigenvectors are useful because they provide a coordinate transformation from the high-dimensional measurement space into a low-dimensional pattern space.They are arranged hierarchically, with $u_1$ being more significant than $u_2$. The U matrix is unitary, which means $U^TU = UU^T =I$. In other words, the transpose of $U$ is the inverse of $U$.

The diagonal elements of Σ are also arranged hierarchically, with the elements ordered as $\sigma_1 \geq \sigma_2 \geq \sigma_m \geq 0$. As the $\sigma$s get smaller they approximate $X$ less, which means the approximation of $X$ is influenced more by the dominate elements of $\sigma$.
       
The first column of $V^T$ tells us how much of a mixture between all of the columns of $U$ that should add up to make a row of $X$. 

In summary of $U\Sigma V^T$:
1. $U$ contains information about the columns space of $X$
2. $V$ contains information about the row space of $X$
3. $\Sigma$ tells us how important the columns of $U$ and $V$ are to $X$

In [1]:
import numpy as np
from numpy import linalg as LA
np.set_printoptions(suppress=True)

In [2]:
A = np.array([[1,-1,3],[3,1,1]])
A

array([[ 1, -1,  3],
       [ 3,  1,  1]])

The $\Sigma$ matrix contains the eigenvalues of $A^TA$

In [3]:
temp = A.T @ A
temp

array([[10,  2,  6],
       [ 2,  2, -2],
       [ 6, -2, 10]])

$$Det(A^TA - \lambda I) = 0 $$

To find singular values we take the square root of all non-zero eigenvalues

In [4]:
w, v = LA.eig(temp)
idx = w.argsort()[::-1] 
w = w[idx]
v = v[:,idx]

The eigenvalue 16 corresponds to the eigenvector [ 0.70710678  0.57735027 -0.40824829]

In [5]:
# the first eigenvalue in the array
print(f"{w[0]:.0f}")

16


In [6]:
# the eigenvector associated with the first eigenvalue
print(v[:,0])

[-0.70710678 -0.         -0.70710678]


In [7]:
# The eigenvalues
w

array([16.,  6., -0.])

<b> $\Sigma$ contains the square roots of the eigenvalues </b>

In [8]:
sigma = np.sqrt(w)
sigma = np.nan_to_num(sigma)
sigma = np.sort(sigma)
sigma = sigma[::-1]
sigma = np.round(sigma,2)

sigma

C:\Users\coryb\AppData\Local\Temp\ipykernel_15124\945988442.py:1: RuntimeWarning: invalid value encountered in sqrt
  sigma = np.sqrt(w)


array([4.  , 2.45, 0.  ])

In [9]:
Sigma = np.diag(sigma)[:-1]
print(Sigma)


[[4.   0.   0.  ]
 [0.   2.45 0.  ]]


<b> The eigenvectors of $A^TA$ are transposed to form $V^T$ </b>

In [10]:
# eigenvectors
V = v.T
V

array([[-0.70710678, -0.        , -0.70710678],
       [-0.57735027, -0.57735027,  0.57735027],
       [ 0.40824829, -0.81649658, -0.40824829]])

We know that $A = U \Sigma V^T$. With some manipulation, we can rearrange the values to assist in finding $U$.
1. $A = U \Sigma V^T$
2. $AV = U \Sigma V^T V$
3. $AV = U \Sigma I$
4. $AV = U \Sigma$

Therefore, 
$$A \vec{v}_1 = \sigma_1 \vec{u}_1$$
$$A \vec{v}_2 = \sigma_2 \vec{u}_2$$

Further, the first column of $U$ is found by 

$$ u_1 = \frac{1}{\sigma_1}A\vec{v}_1 $$

In [11]:
u_1 = 1 / Sigma[0,0] * A @V[0].reshape(-1,1)
u_1

array([[-0.70710678],
       [-0.70710678]])

In [12]:
u_2 = 1 / Sigma[1,1] * A @V[1].reshape(-1,1)
u_2

array([[ 0.70695951],
       [-0.70695951]])

In [13]:
U = np.concatenate((u_1, u_2), axis=1)
U

array([[-0.70710678,  0.70695951],
       [-0.70710678, -0.70695951]])

### Inspect the decomposition by multiplying $U \Sigma V^T$ together. We should approximate $X$ very closely.

In [14]:
Final_Test = U@ Sigma @V
Final_Test

array([[ 1., -1.,  3.],
       [ 3.,  1.,  1.]])

Now SVD will be done using the built-in functions in numpy

In [15]:
[u_python, s_python, v_python] = np.linalg.svd(A)

In [16]:
u_python

array([[ 0.70710678, -0.70710678],
       [ 0.70710678,  0.70710678]])

In [17]:
s_python

array([4.        , 2.44948974])

In [18]:
v_python

array([[ 0.70710678, -0.        ,  0.70710678],
       [ 0.57735027,  0.57735027, -0.57735027],
       [-0.40824829,  0.81649658,  0.40824829]])